#### Day 10: Subqueries in SQL 🎉

Welcome to Day 10! Today, we’ll dive into **subqueries** — a powerful way to nest one query within another. Subqueries are essential for writing advanced and efficient SQL queries.

#### 1. Database Connection

In [ ]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="30_Days_SQL"
)
cursor = conn.cursor()
print("Connected to '30_Days_SQL'!")

---
#### 1. Subquery in the WHERE Clause 🔎
Find employees who earn more than the average salary.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS employees")
cursor.execute('''
CREATE TABLE employees (
    employee_id INT PRIMARY KEY AUTO_INCREMENT,
    employee_name VARCHAR(100),
    salary INT,
    department_id INT
)
''')

employees_data = [
    ('John', 75000, 1),
    ('Sarah', 80000, 1),
    ('Mike', 60000, 2),
    ('Anna', 55000, 2),
    ('Tom', 90000, 1)
]
cursor.executemany("INSERT INTO employees (employee_name, salary, department_id) VALUES (%s, %s, %s)", employees_data)
conn.commit()

query = """
SELECT employee_name, salary
FROM employees
WHERE salary > (SELECT AVG(salary) FROM employees);
"""
pd.read_sql(query, conn)

#### 2. Subquery in the SELECT Clause ⚖️
Find each employee’s salary and the average salary of their department.

In [ ]:
query = """
SELECT employee_name, salary,
       (SELECT AVG(salary)
        FROM employees e2
        WHERE e1.department_id = e2.department_id) AS avg_department_salary
FROM employees e1;
"""
pd.read_sql(query, conn)

#### 3. Subquery in the FROM Clause 🔄
Find the department total salaries (using IT as an example label).

In [ ]:
query = """
SELECT department_id, total_salary
FROM (SELECT department_id, SUM(salary) AS total_salary
      FROM employees
      GROUP BY department_id) AS department_totals
WHERE total_salary = (SELECT MAX(total_salary)
                      FROM (SELECT department_id, SUM(salary) AS total_salary
                            FROM employees
                            GROUP BY department_id) AS department_totals);
"""
pd.read_sql(query, conn)

In [ ]:
conn.close()